In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm

def calculate_feature(image_path):
    """
    计算图像的特征向量。

    参数：
        image_path (str): 图像文件的路径。

    返回：
        np.ndarray: 图像的特征向量。
    """
    # 将图像路径转换为Unicode字符串
    image_path_unicode = image_path.encode("utf-8").decode("utf-8")
    
    # 读取图像
    img = cv2.imread(image_path_unicode, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None

    # 提取图像特征
    orb = cv2.ORB_create()
    keypoints, descriptors = orb.detectAndCompute(img, None)

    # 如果没有提取到特征，则返回 None
    if descriptors is None:
        return None
    
    # 对特征向量求均值，得到图像的特征向量
    feature_vector = np.mean(descriptors, axis=0)
    return feature_vector


def remove_duplicate_images(image_dir, remove_from_disk=False):
    """
    从给定目录中去除重复的图像文件，并返回唯一图像的路径列表。

    参数：
        image_dir (str): 包含图像文件的目录路径。
        remove_from_disk (bool): 是否从硬盘上删除重复的图像文件，默认为 False。

    返回：
        list: 唯一图像文件的路径列表。
    """
    # 获取目录中的所有图像文件路径
    image_paths = [os.path.join(image_dir, file) for file in os.listdir(image_dir) if file.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    # 输出目录中的文件数
    print(f"目录 {image_dir} 中的文件数: {len(image_paths)}")

    # 初始化唯一图像路径列表和已处理的图像特征列表
    unique_images = []
    processed_features = []

    # 遍历图像文件
    print("开始处理图像...")
    for i, image_path in enumerate(tqdm(image_paths, desc="Processing")):
        try:
            # 输出序号和图片路径
            print(f"处理第 {i+1} 张图片: {image_path}")
            
            # 计算图像的特征向量
            feature_vector = calculate_feature(image_path)
            
            # 如果特征向量为 None，则记录日志并跳过该图像
            if feature_vector is None:
                log_message = f"警告: 图像 {image_path} 的特征向量为空，已跳过该图像."
                print(log_message)
                continue
            
            # 如果特征向量已经存在于已处理的特征列表中，则认为是重复图像
            if any(np.allclose(feature_vector, feature) for feature in processed_features):
                # 记录日志和输出重复图片信息
                log_message = f"重复图片: {image_path}, 特征值: {feature_vector}"
                print(log_message)
                if remove_from_disk:
                    # 如果需要删除重复图像，则执行删除操作
                    os.remove(image_path)
                continue
            
            # 将图像的特征向量添加到已处理的特征列表中，并将图像路径添加到唯一图像列表中
            processed_features.append(feature_vector)
            unique_images.append(image_path)
        except Exception as e:
            # 捕获并输出异常信息
            log_message = f"错误: 处理图像 {image_path} 时出错: {e}"
            print(log_message)
    
    # 如果唯一图像列表为空，输出提示信息
    if not unique_images:
        print("未找到任何唯一图像.")
    else:
        print(f"共找到 {len(unique_images)} 张唯一图像.")
    
    return unique_images

# 主方法，用于外部调用
def main(image_dir, remove_from_disk=False):
    """
    对图像目录中的图像进行去重。

    参数：
        image_dir (str): 包含图像的目录路径。
        remove_from_disk (bool): 如果为True，则从磁盘中删除重复图像。

    返回：
        unique_images_list (list): 不重复的图像文件路径列表。
    """
    # 调用去除重复图像的函数
    unique_images_list = remove_duplicate_images(image_dir, remove_from_disk)
    
    # 返回不重复的图像列表
    return unique_images_list

In [2]:
# 调用函数并接收返回的去重后文件列表
unique_images_list = main("F:\\pic\\zhytest", False)

目录 F:\pic\zhytest 中的文件数: 49
开始处理图像...


Processing:   0%|          | 0/49 [00:00<?, ?it/s]

Processing:   2%|▏         | 1/49 [00:00<00:07,  6.76it/s]

处理第 1 张图片: F:\pic\zhytest\_storage_emulated_0_sina_weibo_storage_photoalbum_save_weibo_img-a0d23d3cc2497bf45a3882a168a52c9f.jpg
处理第 2 张图片: F:\pic\zhytest\5b2408a2548041ba8b6200e61da64d01.jpg


Processing:   6%|▌         | 3/49 [00:00<00:04, 11.47it/s]

...

处理第 44 张图片: F:\pic\zhytest\cf1b9d16fdfaaf519ee797ff8c5494eef01f7a6d.jpg
处理第 45 张图片: F:\pic\zhytest\ChMlWl1omuyIf0S1AAlctLjmfdIAAM4mwH7q1YACVzM700.jpg
处理第 46 张图片: F:\pic\zhytest\72666D36E7A45ECF5D97E3E2A94981A5.jpg


Processing:  96%|█████████▌| 47/49 [00:04<00:00, 13.24it/s]

处理第 47 张图片: F:\pic\zhytest\551433992CB7682A291A666534355A22.jpg
处理第 48 张图片: F:\pic\zhytest\69192ab4jw1e8yvrrpz7oj20tm18gwkf.jpg
处理第 49 张图片: F:\pic\zhytest\6244181ejw1ebo5swe4m1j218g0szdqj.jpg


Processing: 100%|██████████| 49/49 [00:04<00:00, 11.99it/s]

共找到 32 张唯一图像.
